In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

training_dir='/content/drive/My Drive/datax/project'
testing_dir='/content/drive/My Drive/data/test'

training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
      )

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	training_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=128
)

validation_generator = validation_datagen.flow_from_directory(
	testing_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=128
)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(include_top=False,
                                weights='imagenet',
                                input_shape=(224,224,3))
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=20, validation_data = validation_generator, verbose = 1, callbacks=[callbacks])
model.save('/content/drive/My Drive/ts.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
import pandas as pd
pesticide={
'pest':['asiatic rice borer','black cutworm','mole cricket','paddy stem maggot','red spider','rice gall                     midge','rice leafhopper','rice water weevil','small brown plant hopper','wireworm'],
'remedy':['Spray products containing chlorantraniliprole','Products containing chlorpyrifos, beta-methrin,               delta methrin','Spray Sevin Insect Killer','Spray Cartap Hydrochloride','Spray wettable sulphur 50','Spray        Phosalone','Spray Phosphamidon','Spray Lambda-cyhalothrin, diflubenzuron','Spray buprofezin, pymetrozine',       'Spray Pyrethrin']
}

df=pd.DataFrame(pesticide,columns=['pest','remedy'],index=[0,1,2,3,4,5,6,7,8,9])
print(df)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
 model=load_model()
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x=x/255.0
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=128)
  print(fn)
  print(classes)

In [ ]:
model=load_model('/content/drive/My Drive/ts.h5')
uploaded = files.upload()
import pandas as pd
pesticide={
    'pest':['asiatic rice borer','black cutworm','mole cricket','paddy stem maggot','red spider','rice gall midge','rice leafhopper','rice water weevil','small brown plant hopper','wireworm'],
    'remedy':['Spray products containing chlorantraniliprole','Products containing chlorpyrifos, beta-methrin, delta methrin','Spray Sevin Insect Killer','Spray Cartap Hydrochloride','Spray wettable sulphur 50','Spray Phosalone','Spray Phosphamidon','Spray Lambda-cyhalothrin, diflubenzuron','Spray buprofezin, pymetrozine','Spray Pyrethrin']
}

df=pd.DataFrame(pesticide,columns=['pest','remedy'],index=[0,1,2,3,4,5,6,7,8,9])
for fn in uploaded.keys():
  path = fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x=x/255.0
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=128)
  print(classes)
  print('pest:')
  print(df.loc[np.argmax(classes)]['pest'])
  print('Pesticide:')
  print(df.loc[np.argmax(classes)]['remedy'])